# Sample-based quantum diagonalization of a chemistry Hamiltonian
https://quantum.cloud.ibm.com/docs/en/tutorials/sample-based-quantum-diagonalization

In [7]:
import pyscf
import pyscf.cc
import pyscf.mcscf
import ffsim
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import qiskit_ibm_runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

import qiskit_addon_sqd

from typing import Sequence
import rustworkx
from rustworkx import PyGraph, NoEdgeBetweenNodes
from qiskit.providers import BackendV2
import pickle

print('pyscf version:', pyscf.__version__)
print('Qiskit version: ', qiskit.__version__)
print('Qiskit Runtime version: ', qiskit_ibm_runtime.__version__)

import importlib.metadata
print('qiskit_addon_sqd version: ', importlib.metadata.version('qiskit_addon_sqd'))
print('ffsim:', importlib.metadata.version('ffsim'))

pyscf version: 2.10.0
Qiskit version:  2.2.1
Qiskit Runtime version:  0.42.0
qiskit_addon_sqd version:  0.12.0
ffsim: 0.0.60


# 고전 화학 계산

In [8]:
open_shell = False # Closed Shell: 최종 솔루션에서 RHF 그라운드 스테이트가 가장 큰 비중을 차지함
spin_sq = 0 # 총 스핀 제곱(S^2)이 0인 Singlet state

# gto: Gaussian type orbitals
# Mole() : 분자 객체
mol = pyscf.gto.Mole() 

# N2 분자 생성
mol.build(
    # 질소 원자의 위치 x축 1 옹스트롬
    atom=[["N", (0, 0, 0)], ["N", (1.0, 0, 0)]], 
    
    # cc-pvdz: 기저함수, sto-3g보다 정확도 높은 표준 기저 함수 중 하나
    basis="cc-pvdz", 
    
    # D-infinitiy-h 대칭성 :계산을 효율적으로 하게 함
    # D: 주축(x축) 
    # infinite: 축을따라 임의의 각도로 돌려도 대칭임
    # h: horizontal, N끼리 거울면 대칭
    symmetry="Dooh", 
)

# Active Space 정의
# 가장 에너지 낮은 2개의 sptial orbital 동결 (4개의 큐빗 아낌)
# 공간 오비탈 28 -> 26 (큐빗 56 -> 52)
n_frozen = 2
active_space = range(n_frozen, mol.nao_nr()) # mol.nao_nr(): 전체 공간 오비탈의 수
 
# 생성된 mol 기반으로 RHF(restrict Hartree-Fock) 계산: 분자 오비탈 계산
# scf.mo_coeff: 계산된 분자 오비탈 (MO)이 어떤 원자 오비탈(AO)의 조합으로 이루어져 있는지 나타내는 계수 행렬
# scf.mo_energy: 각 분자 오비탈의 에너지
# scf.mo_occ: 각 분자 오비탈에 전자가 몇 개 채워졌는지 (0.0 또는 2.0) 나타냄 (낮은 에너지부터 채워짐)
# Output: RHF 에너지(하나의 Slater Determinant로 근사), converged SCF energy = ...
scf = pyscf.scf.RHF(mol).run() 

num_orbitals = len(active_space) # 26: 낮은 에너지부터 0번
n_electrons = int(sum(scf.mo_occ[active_space])) # 총 전자 개수 = 10

# Open Shell 이면 계산 달라짐
num_elec_a = (n_electrons + mol.spin) // 2 # 업스핀 수
num_elec_b = (n_electrons - mol.spin) // 2 # 다운스핀 수

# MCSCF(Multi-Configurational Self-Consistent Field) (RHF, 또는 SCF) 방법보다 더 정확하고 강력한 상위 레벨의 양자 화학 계산법
# CASCI (Complete Active Space CI): MCSCF의 한 종류
# CASCI (Complete Active Space Configuration Interaction, 완전 활성 공간 배치 상호작용) 기능을 사용하기 위한 객체(cas) 생성
cas = pyscf.mcscf.CASCI(scf, num_orbitals, (num_elec_a, num_elec_b))

mo = cas.sort_mo(active_space, base=0) # 활성 공간 오비탈 골라내고 정렬, 활성오비탈 0번부터 시작

# 1전자 적분 h_pq 계산
# nuclear_repulsion_energy: 핵과 핵 사이의 반발 에너지입, 분자가 고정되어 있으므로 이 값은 상수
hcore, nuclear_repulsion_energy = cas.get_h1cas(mo)

# cas.get_h2cas(mo): 2전자 적분 (pq|rs) 계산 -> 1차원 배열로 반환
# pyscf.ao2mo.restore: 1차원 배열을 4차원 텐서(26 26 26 26)로 복원
# 1: 4차원 텐서로 복원하라는 옵션
# num_orbitals: 복원할 텐서의 크기
eri = pyscf.ao2mo.restore(1, cas.get_h2cas(mo), num_orbitals) 

# 26개 오비탈 문제를 고전 컴퓨터에서 정밀한 방식(SCI)으로 미리 계산해 둔 벤치마크 값
exact_energy = -109.22690201485733

converged SCF energy = -108.929838385609


In [9]:
# LUCJ 회로에 사용할 파라미터를 고전 컴퓨터로 계산
# pyscf.cc.CCSD(...): PySCF의 CCSD(coupled cluster singles and doubles) 계산 준비
# scf: reference state로 rhf 계산 결과를 참조
# frozen: 동결시킬 오비탈
# Output: CCSD 방법으로 근사한 그라운드 스테이트 에너지와 E_ccsd - E_RHF 값
ccsd = pyscf.cc.CCSD(
    scf, frozen=[i for i in range(mol.nao_nr()) if i not in active_space]
).run()

# t1, t2가 K, J 계수들을 만들기 위한 재료
t1 = ccsd.t1
t2 = ccsd.t2

E(CCSD) = -109.2177884185543  E_corr = -0.2879500329450035


# 양자 샘플링

In [10]:
# ffsim과 qiskit을 이용한 양자 회로 설계

n_reps = 1 # 회로 레이어

# 텀들중에서 ibm의 heavy-hex 구조에 맞는 상호작용만 남김
# Gemini:
 # (p, p+1), (p, p) 도 서로 에너지가 가까운 오비탈이지만, 그렇다고 계산에 반드시 중요한 텀이라는것은 아님 
 # 이 또한 ibm topology에 맞추기 위한것
 # 중요성을 t2 진폭의 크기를 기준으로 판단하는게 나을것
alpha_alpha_indices = [(p, p + 1) for p in range(num_orbitals - 1)] # 같은 스핀끼리 상호작용 쌍 정의
alpha_beta_indices = [(p, p) for p in range(0, num_orbitals, 4)] # 다른 스핀끼리 상호작용 쌍 정의
 
# ucj_op: UCJ 회로를 나타내는 operator 생성
# UCJOpSpinBalanced: 스핀 밸런스드 -> J나 K에 스핀 인덱스가 없음, 업스핀 다운스핀이 같음
# from_t_amplitudes: 이 연산자들을 t 진폭으로부터 만든다
ucj_op = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
    t2=t2,
    t1=t1,
    n_reps=n_reps,
    interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
    # t2 텐서를 최적화(압축)하고, 압축된 t2를 통해 K와 J를 계산함
    optimize=True,
    # t2 텐서 최적화를 1000번 안쪽으로만 함
    options=dict(maxiter=1000),
)
 
nelec = (num_elec_a, num_elec_b)

# create an empty quantum circuit
qubits = QuantumRegister(2 * num_orbitals, name="q")
circuit = QuantumCircuit(qubits)

# 하트리폭 상태를 만듬 |000...0111...1 000...0111...1>
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), qubits)
# apply the UCJ operator to the reference state
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)
circuit.measure_all()
# display(circuit.draw('mpl'))

In [11]:
QiskitRuntimeService.save_account(
    token="Kj0pgEv4yf6BzJtENOZO5VQuqwXpahRSOHBgd0ifGX_Y",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b62848de09854d1f93f068e55909ba0f:13ae3caf-3da8-4992-9d4e-d6ae3a552a4f::",
    overwrite=True,
    set_as_default=True
)

service = QiskitRuntimeService()
print(service.backends())
backend_name = "ibm_fez"
backend = service.backend(backend_name)
print(backend)

management.get:WARNING:2025-11-22 16:30:59,134: Loading default saved account


[<IBMBackend('ibm_fez')>, <IBMBackend('ibm_torino')>, <IBMBackend('ibm_marrakesh')>]
<IBMBackend('ibm_fez')>


In [12]:
##############################추가 시작##############################
IBM_TWO_Q_GATES = {"cx", "ecr", "cz"}


def _create_linear_chains(num_orbitals: int) -> PyGraph:
    """스핀-업 체인 하나, 스핀-다운 체인 하나: 총 2*num_orbitals 노드를 가진 두 개의 직선 체인 그래프."""
    G = PyGraph()

    # alpha 체인: 0 .. num_orbitals-1
    for n in range(num_orbitals):
        G.add_node(n)
    for n in range(num_orbitals - 1):
        G.add_edge(n, n + 1, None)

    # beta 체인: num_orbitals .. 2*num_orbitals-1
    for n in range(num_orbitals, 2 * num_orbitals):
        G.add_node(n)
    for n in range(num_orbitals, 2 * num_orbitals - 1):
        G.add_edge(n, n + 1, None)

    return G


def _create_lucj_zigzag_layout(
    num_orbitals: int, backend_coupling_graph: PyGraph
) -> tuple[PyGraph, int]:
    """
    LUCJ에 맞는 '가상' 지그재그 그래프를 만들고,
    이 그래프가 백엔드 coupling graph의 서브그래프가 되도록 브리지(α-β) 개수를 조정한다.
    반환값:
        G_new: 백엔드와 호환 가능한 지그재그 그래프
        num_alpha_beta_qubits: 사용된 브리지(α-β) 노드 수
    """
    isomorphic = False
    base_graph = _create_linear_chains(num_orbitals)
    num_iters = num_orbitals  # 최대 num_orbitals/4 개 지점까지 브리지 시도

    while not isomorphic and num_iters > 0:
        G_new = base_graph.copy()
        num_alpha_beta_qubits = 0

        # p = 0, 4, 8, ... 에서 브리지 노드를 추가.
        # 단, n < num_iters 로 제한을 둬서 브리지 개수를 줄여가며 시도.
        for n in range(num_iters):
            if n % 4 == 0:
                new_node = 2 * num_orbitals + num_alpha_beta_qubits
                G_new.add_node(new_node)
                # alpha-체인 쪽과 연결
                G_new.add_edge(n, new_node, None)
                # beta-체인 쪽과 연결
                G_new.add_edge(n + num_orbitals, new_node, None)
                num_alpha_beta_qubits += 1

        isomorphic = rustworkx.is_subgraph_isomorphic(
            backend_coupling_graph, G_new
        )
        num_iters -= 1

    return G_new, num_alpha_beta_qubits


def _lightweight_layout_error_scoring(
    backend: BackendV2,
    virtual_edges: Sequence[Sequence[int]],
    physical_layouts: Sequence[list[int]],
    two_q_gate_name: str,
) -> list[tuple[list[int], float]]:
    """
    가능한 zigzag 레이아웃들(서브그래프 embedding) 중에서,
    2Q gate error + readout error 합이 가장 작은 레이아웃을 고르기 위한 점수 계산.
    """
    props = backend.properties()
    scores: list[tuple[list[int], float]] = []

    for layout in physical_layouts:
        total_2q_error = 0.0
        for u, v in virtual_edges:
            phys_edge = (layout[u], layout[v])
            try:
                ge = props.gate_error(two_q_gate_name, phys_edge)
            except Exception:
                # 방향 반대일 수도 있으니 한번 더 시도
                ge = props.gate_error(two_q_gate_name, phys_edge[::-1])
            total_2q_error += ge

        total_meas_error = 0.0
        for q in layout:
            total_meas_error += props.readout_error(q)

        scores.append((layout, total_2q_error + total_meas_error))

    # 점수가 작은(= 노이즈가 적은) 순으로 정렬
    scores.sort(key=lambda x: x[1])
    return scores


def _make_backend_cmap_pygraph(backend: BackendV2) -> PyGraph:
    """
    Backend의 CouplingMap을 무방향 PyGraph로 바꾸고, 중복 edge 제거.
    """
    graph = backend.coupling_map.graph
    if not graph.is_symmetric():
        graph.make_symmetric()

    backend_graph = graph.to_undirected()
    seen_edges: set[tuple[int, int]] = set()

    for u, v in list(backend_graph.edge_list()):
        key = tuple(sorted((u, v)))
        if key in seen_edges:
            try:
                backend_graph.remove_edge(u, v)
            except NoEdgeBetweenNodes:
                pass
        else:
            seen_edges.add(key)

    return backend_graph


def get_zigzag_physical_layout(
    num_orbitals: int, backend: BackendV2, score_layouts: bool = True
) -> tuple[list[int], int]:
    """
    LUCJ용 지그재그 패턴을 가진 물리 큐빗 레이아웃을 생성.
    반환:
        initial_layout: 길이 2*num_orbitals 인 리스트 (각 논리 큐빗 → 물리 큐빗 인덱스)
        num_alpha_beta_qubits: 브리지(α-β 상호작용용) 노드 수
    """
    backend_graph = _make_backend_cmap_pygraph(backend)

    # 1) 백엔드와 호환되는 가상 지그재그 그래프 만들기
    zigzag_graph, num_alpha_beta_qubits = _create_lucj_zigzag_layout(
        num_orbitals=num_orbitals,
        backend_coupling_graph=backend_graph,
    )

    # 2) 해당 그래프가 백엔드 그래프의 어떤 서브그래프로 들어가는지 모든 매핑 찾기
    mappings = list(
        rustworkx.vf2_mapping(
            backend_graph,
            zigzag_graph,
            subgraph=True,
        )
    )

    virtual_edges = list(zigzag_graph.edge_list())
    layouts: list[list[int]] = []

    # rustworkx.vf2_mapping 은 {backend_node: virtual_node} 딕셔너리들을 반환.
    for mapping in mappings:
        layout = [None] * (2 * num_orbitals + num_alpha_beta_qubits)
        for backend_node, virtual_node in mapping.items():
            layout[virtual_node] = backend_node
        layouts.append(layout)

    # 백엔드의 2Q 게이트 이름 선택 (cx/ecr/cz 중 하나)
    two_q_gate_name = IBM_TWO_Q_GATES.intersection(
        backend.configuration().basis_gates
    ).pop()

    if score_layouts:
        scores = _lightweight_layout_error_scoring(
            backend=backend,
            virtual_edges=virtual_edges,
            physical_layouts=layouts,
            two_q_gate_name=two_q_gate_name,
        )
        best_layout, _ = scores[0]
    else:
        best_layout = layouts[0]

    # 맨 뒤 num_alpha_beta_qubits 개는 "브리지 노드"에 해당 → 스핀오비탈용 논리 큐빗만 남김
    return best_layout[:-num_alpha_beta_qubits], num_alpha_beta_qubits
##############################추가 끝##############################

##############################교체 시작##############################
# 1) 지그재그 물리 레이아웃 계산
initial_layout, num_alpha_beta = get_zigzag_physical_layout(
    num_orbitals=num_orbitals,
    backend=backend,
)

print("Initial zigzag layout:", initial_layout)
print("Number of alpha-beta bridge nodes (unused ancilla qubits):", num_alpha_beta)

# 2) 이 레이아웃을 사용하는 pass manager 생성
pm = generate_preset_pass_manager(
    backend=backend,
    optimization_level=3,
    initial_layout=initial_layout,
)

# 3) ffsim PRE_INIT 패스를 pre_init 단계에 붙여서 LUCJ 회로를 더 잘 줄이기
pm.pre_init = ffsim.qiskit.PRE_INIT

# 4) 회로 트랜스파일
isa_circ = pm.run(circuit)

# 5) 샘플러로 실행
sampler = Sampler(mode=backend)
job = sampler.run([isa_circ], shots=100_000)
print(f">>> Job ID: {job.job_id()}")
##############################교체 끝##############################

Initial zigzag layout: [13, 12, 11, 18, 31, 30, 29, 38, 49, 48, 47, 57, 67, 66, 65, 77, 85, 84, 83, 96, 103, 102, 101, 116, 121, 120, 15, 19, 35, 34, 33, 39, 53, 52, 51, 58, 71, 70, 69, 78, 89, 88, 87, 97, 107, 106, 105, 117, 125, 124, 123, 136]
Number of alpha-beta bridge nodes (unused ancilla qubits): 7
>>> Job ID: d4gmdf4cdebc73f19sb0


In [13]:
job = service.job('d4gmdf4cdebc73f19sb0')
result = job.result()
print(dir(result))
#print(dir(result[0].data))
count = result[0].data.meas
#print(type(count))
count

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', '_metadata', '_pub_results', 'metadata']


BitArray(<shape=(), num_shots=100000, num_bits=52>)

# Configuration Recovery 및 결과

In [14]:
from functools import partial

from qiskit_addon_sqd.fermion import (
    SCIResult,
    diagonalize_fermionic_hamiltonian,
    solve_sci_batch,
)

# 추가: BitArray → 배열 변환 + Hamming postselection
from qiskit_addon_sqd.counts import bit_array_to_arrays
from qiskit_addon_sqd.subsampling import postselect_by_hamming_right_and_left

# SQD 이터레이션 중지 조건
energy_tol = 1e-3          # 에너지 변화
occupancies_tol = 1e-3     # 오비탈 점유율 n, ( n_(i+1)-n_(i) )의 infinite norm으로 정의
max_iterations = 5         # 최대 반복 횟수

# Batch와 대각화 관련 옵션
num_batches = 3 # Batch의 수 (K)
samples_per_batch = 300 # Batch의 크기
symmetrize_spin = True # 스핀 대칭성 고려 결과 개선
# configuration recovery를 위한 샘플링때 계수가 큰 bitstring들을 다음 이터레이션을 위한 샘플링에 추가
carryover_threshold = 1e-4 
max_cycle = 200 # Davidson 알고리즘등의 반복을 200번으로 제한

# spin_sq = 0.0: 스핀 symmetry 관련 패널티항 추가 
# H + lambda(S^2 + s(s+1))^2 여기서 s(s+1)=0 을 넣는다는 뜻
sci_solver = partial(solve_sci_batch, spin_sq=0.0, max_cycle=max_cycle)
result_history = []

# 결과 출력용 함수
def callback(results: list[SCIResult]):
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + nuclear_repulsion_energy}")
        print(
            f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}"
        )

# --- HF 기반 initial n: 노이즈 심해서 유효 샘플이 0개일 때만 쓸 예정 ---
initial_occupancies_spin_orbital = np.zeros(2 * num_orbitals)
initial_occupancies_spin_orbital[:num_elec_a] = 1.0
initial_occupancies_spin_orbital[
    num_orbitals : num_orbitals + num_elec_b
] = 1.0

# --- 1) BitArray에서 raw 샘플 개수 / 유효 샘플 개수 계산 ---

# BitArray(count) -> bitstring_matrix (bool), probabilities
bitstring_matrix, probabilities = bit_array_to_arrays(count)

num_raw_samples = bitstring_matrix.shape[0]
print(f"[SQD] Total raw samples from hardware: {num_raw_samples}")

# 비트스트링 형식: [b_N, ..., b_0, a_N, ..., a_0]
# → 오른쪽 절반(α)의 Hamming weight = num_elec_a
#   왼쪽 절반(β)의 Hamming weight = num_elec_b
bitstring_matrix_valid, probabilities_valid = postselect_by_hamming_right_and_left(
    bitstring_matrix,
    probabilities,
    hamming_right=num_elec_a,
    hamming_left=num_elec_b,
)

num_valid_samples = bitstring_matrix_valid.shape[0]
print(
    f"[SQD] Valid samples after (Nα={num_elec_a}, Nβ={num_elec_b}) postselection: "
    f"{num_valid_samples}"
)

# --- 2) initial_occupancies 선택 로직 ---

if num_valid_samples == 0:
    print(
        "[SQD] WARNING: No valid samples found. "
        "Falling back to HF-based initial occupancies."
    )
    initial_occupancies_for_sqd = initial_occupancies_spin_orbital
else:
    print(
        "[SQD] Using tutorial-style SQD: "
        "no explicit initial_occupancies (first n is inferred from samples)."
    )
    initial_occupancies_for_sqd = None

# --- 3) SQD 실행 ---

result = diagonalize_fermionic_hamiltonian(
    hcore,                      # 1-전자 적분 (활성 공간)
    eri,                        # 2-전자 적분 (활성 공간)
    count,                      # 양자 측정 결과 (BitArray 객체)
    samples_per_batch=samples_per_batch,    # 배치당 샘플 수 (300)
    norb=num_orbitals,          # 활성 공간 오비탈 수 (26)
    nelec=nelec,                # 활성 공간 전자 수 ((5, 5))
    num_batches=num_batches,    # 배치 수 (3)
    energy_tol=energy_tol,      # 에너지 수렴 기준 (0.001)
    occupancies_tol=occupancies_tol,  # n 수렴 기준 (0.001), l maximum norm
    max_iterations=max_iterations,    # 최대 반복 횟수 (5)
    sci_solver=sci_solver,      # 계산 함수 위에서 partial 통해서 정의
    symmetrize_spin=symmetrize_spin,    # 스핀 inversion symmetry 고려 방법 사용 (sqrt(d)/2개 뽑아서)
    carryover_threshold=carryover_threshold,    # 중요 배치 유지 기준 (1e-4)
    callback=callback,                          # 반복 시 호출될 함수
    seed=12345,                                 # 재현성을 위한 랜덤 시드
    # initial n: 설정조건부로 HF / None 선택
    initial_occupancies=initial_occupancies_for_sqd,
)

file_name = 'sqd_result3.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(result, f)

print(f"계산 결과가 '{file_name}' 파일로 저장되었습니다.")

[SQD] Total raw samples from hardware: 100000
[SQD] Valid samples after (Nα=5, Nβ=5) postselection: 129
[SQD] Using tutorial-style SQD: no explicit initial_occupancies (first n is inferred from samples).
Iteration 1
	Subsample 0
		Energy: -109.01954808446882
		Subspace dimension: 60025
	Subsample 1
		Energy: -109.01954808446882
		Subspace dimension: 60025
	Subsample 2
		Energy: -109.01954808446882
		Subspace dimension: 60025
Iteration 2
	Subsample 0
		Energy: -109.13329482287051
		Subspace dimension: 301401
	Subsample 1
		Energy: -109.09959073692133
		Subspace dimension: 295936
	Subsample 2
		Energy: -109.12404112448922
		Subspace dimension: 300304
Iteration 3
	Subsample 0
		Energy: -109.16124450459938
		Subspace dimension: 470596
	Subsample 1
		Energy: -109.16935890275445
		Subspace dimension: 462400
	Subsample 2
		Energy: -109.16032884866283
		Subspace dimension: 444889
Iteration 4
	Subsample 0
		Energy: -109.17764554385688
		Subspace dimension: 632025
	Subsample 1
		Energy: -109.182

# Ext-SQD

# 계수 크기가 $<10^{-3}$인 basis 제거

In [12]:
import numpy as np

# 1. 임계값 설정 (논문 기준: 1e-3)
truncation_threshold = 1e-2

# 2. 최적의 결과 선택
best_result = result
best_state = best_result.sci_state

print(f"Best Batch Energy: {best_result.energy + nuclear_repulsion_energy}")
# amplitudes는 (Alpha String 개수 x Beta String 개수)의 2차원 행렬입니다.
print(f"Original Subspace Shape (M x N): {best_state.amplitudes.shape}")

# 3. 마스크 생성 및 인덱스 추출
# 진폭의 절대값이 임계값 이상인 위치(행, 열)를 찾습니다.
mask = np.abs(best_state.amplitudes) >= truncation_threshold
rows, cols = np.where(mask)

# 4. 상태 필터링 (Truncation)
# 해당하는 진폭값들을 1차원 배열로 추출합니다.
filtered_amplitudes = best_state.amplitudes[rows, cols]

# 행 인덱스는 Alpha String, 열 인덱스는 Beta String에 대응됩니다.
# (속성 이름 수정: alpha_strings -> ci_strs_a)
filtered_alpha_strings = best_state.ci_strs_a[rows]
filtered_beta_strings = best_state.ci_strs_b[cols]

print(f"Truncated Subspace Dimension (Number of configs): {len(filtered_amplitudes)}")

# 5. 정규화 (Normalization)
norm = np.linalg.norm(filtered_amplitudes)
normalized_amplitudes = filtered_amplitudes / norm

print(f"Vector Norm after truncation & normalization: {np.linalg.norm(normalized_amplitudes)}")

Best Batch Energy: -109.18138298285575
Original Subspace Shape (M x N): (923, 923)
Truncated Subspace Dimension (Number of configs): 135
Vector Norm after truncation & normalization: 1.0


In [13]:
import numpy as np
from tqdm import tqdm
import itertools

def get_occupied_indices(bitstring_int, num_orbitals):
    """정수형 비트스트링에서 전자가 있는(1인) 오비탈 인덱스를 반환합니다."""
    return [i for i in range(num_orbitals) if (bitstring_int >> i) & 1]

def get_empty_indices(bitstring_int, num_orbitals):
    """정수형 비트스트링에서 전자가 없는(0인) 오비탈 인덱스를 반환합니다."""
    return [i for i in range(num_orbitals) if not ((bitstring_int >> i) & 1)]

def extend_subspace_generalized(alpha_seeds, beta_seeds, num_orbitals):
    """
    주어진 시드 상태들에 대해 일반화된 Singles & Doubles 여기를 적용하여
    확장된 부분 공간(Extended Subspace)의 기저 집합을 생성합니다.
    """
    extended_basis_set = set()
    
    # 초기 시드 상태들을 먼저 집합에 추가
    for a, b in zip(alpha_seeds, beta_seeds):
        extended_basis_set.add((a, b))
    
    print(f"Starting extension from {len(alpha_seeds)} seed configurations...")
    
    # 각 시드 상태에 대해 루프
    for seed_idx in tqdm(range(len(alpha_seeds)), desc="Extending Subspace"):
        curr_a = alpha_seeds[seed_idx]
        curr_b = beta_seeds[seed_idx]
        
        # 1. 현재 상태의 Occupied(전자가 있는 방)와 Virtual(빈 방) 인덱스 파악
        occ_a = get_occupied_indices(curr_a, num_orbitals)
        vir_a = get_empty_indices(curr_a, num_orbitals)
        
        occ_b = get_occupied_indices(curr_b, num_orbitals)
        vir_b = get_empty_indices(curr_b, num_orbitals)
        
        # ---------------------------------------------------------
        # A. Single Excitations (i -> a)
        # ---------------------------------------------------------
        
        # Alpha Singles
        generated_alpha_singles = []
        for i in occ_a:
            for a in vir_a:
                # XOR 연산으로 i번째 비트를 0으로, a번째 비트를 1로 바꿈
                new_a = curr_a ^ (1 << i) ^ (1 << a)
                extended_basis_set.add((new_a, curr_b))
                generated_alpha_singles.append(new_a) # 나중에 Double 만들때 재사용
        
        # Beta Singles
        generated_beta_singles = []
        for i in occ_b:
            for a in vir_b:
                new_b = curr_b ^ (1 << i) ^ (1 << a)
                extended_basis_set.add((curr_a, new_b))
                generated_beta_singles.append(new_b)

        # ---------------------------------------------------------
        # B. Double Excitations 
        # ---------------------------------------------------------
        
        # 1. Alpha-Alpha Doubles (두 개의 Alpha 전자를 이동)
        # (i, j) -> (a, b)
        # itertools.combinations를 써서 중복 없이 순서쌍 생성
        for (i, j) in itertools.combinations(occ_a, 2):
            for (a, b) in itertools.combinations(vir_a, 2):
                # 4개의 비트를 동시에 뒤집음
                mask = (1 << i) | (1 << j) | (1 << a) | (1 << b)
                new_a = curr_a ^ mask
                extended_basis_set.add((new_a, curr_b))
                
        # 2. Beta-Beta Doubles (두 개의 Beta 전자를 이동)
        for (i, j) in itertools.combinations(occ_b, 2):
            for (a, b) in itertools.combinations(vir_b, 2):
                mask = (1 << i) | (1 << j) | (1 << a) | (1 << b)
                new_b = curr_b ^ mask
                extended_basis_set.add((curr_a, new_b))
                
        # 3. Alpha-Beta Mixed Doubles (Alpha 1개, Beta 1개 이동)
        # 앞서 만들어둔 Singles 리스트를 활용하면 계산을 줄일 수 있음
        # (New Alpha from Singles) x (New Beta from Singles) 조합
        for new_a in generated_alpha_singles:
            for new_b in generated_beta_singles:
                extended_basis_set.add((new_a, new_b))

    return extended_basis_set

# --- 실행 코드 ---
num_orbitals = 26 # (10e, 26o)

# 앞서 필터링한 1424개의 상태를 입력으로 넣습니다.
# 시간이 조금 걸릴 수 있습니다 (수천만 번의 연산).
extended_basis_set = extend_subspace_generalized(
    filtered_alpha_strings, 
    filtered_beta_strings, 
    num_orbitals
)

# 결과 변환 (Set -> List -> Numpy Array)
extended_basis_list = list(extended_basis_set)
extended_basis_a = np.array([x[0] for x in extended_basis_list], dtype=object) # or dtype=int64
extended_basis_b = np.array([x[1] for x in extended_basis_list], dtype=object)

print(f"\nOriginal SQD Subspace Size: {len(filtered_alpha_strings)}")
print(f"Final Ext-SQD Subspace Size: {len(extended_basis_set)}")
print(f"Expansion Factor: {len(extended_basis_set) / len(filtered_alpha_strings):.2f}x")

Starting extension from 135 seed configurations...


Extending Subspace: 100%|████████████████████████████████████████████████████████████| 135/135 [00:01<00:00, 125.62it/s]



Original SQD Subspace Size: 135
Final Ext-SQD Subspace Size: 1304931
Expansion Factor: 9666.16x


In [ ]:
from qiskit_addon_sqd.fermion import solve_sci
import numpy as np

# 1. 데이터 타입 정리
final_alpha_strs = extended_basis_a.astype(np.int64)
final_beta_strs = extended_basis_b.astype(np.int64)

print(f"Solving Schrödinger equation in Extended Subspace...")
print(f"Dimension: {len(final_alpha_strs)}")
print(f"Target: Ground State (nroots=1)")

# 2. Ext-SQD 문제 풀이
ext_sqd_result = solve_sci(
    (final_alpha_strs, final_beta_strs),    # 1. 기저 (튜플)
    hcore,                                  # 2. 1-전자 적분
    eri,                                    # 3. 2-전자 적분
    num_orbitals,                           # 4. 오비탈 개수 (여기가 빠졌었습니다!)
    nelec=nelec,            # 전자 수
    nroots=1,               # 바닥 상태 1개
    spin_sq=0.0,            # Singlet state
    max_cycle=100,          # 반복 횟수
    tol=1e-6                # 수렴 오차
)

# 3. 결과 출력
total_energy = ext_sqd_result.energy + nuclear_repulsion_energy
print("-" * 50)
print(f"Ext-SQD Ground State Energy: {total_energy:.8f} Ha")
print("-" * 50)

# 비교
if 'best_result' in locals():
    improvement = (best_result.energy + nuclear_repulsion_energy) - total_energy
    print(f"Improvement over SQD: {improvement:.8f} Ha lower")

Solving Schrödinger equation in Extended Subspace...
Dimension: 1304931
Target: Ground State (nroots=1)
